 # CRF

Import pandas + numpy  and load train + test data

In [0]:
import pandas as pd
import numpy as np

data = pd.read_csv('Train_set.csv', encoding='latin1')

# Change de test sets

test = pd.read_csv('Test_set.csv')
# test = pd.read_csv('Zero-shot.csv')

Make a list of all words in the train and test set

In [0]:
words = list(set(data['Word'].values))
test_words = list(set(test["Word"].values))

In [0]:
test_n_words = len(test_words)
n_words = len(words)

Sentence getter

In [0]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [0]:
getter = SentenceGetter(data)
test_getter = SentenceGetter(test)

In [0]:
sent = getter.get_next()
test_sent = test_getter.get_next()

In [0]:
sentences = getter.sentences
test_sentences = test_getter.sentences

Get a word's features

In [0]:
def word2features(sent, i):
    word = str(sent[i][0])
    postag = str(sent[i][1])

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = str(sent[i-1][0])
        postag1 = str(sent[i-1][1])
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = str(sent[i+1][0])
        postag1 = str(sent[i+1][1])
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [0]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
test_X = [sent2features(s) for s in test_sentences]
test_y = [sent2labels(s) for s in test_sentences]

Define Train and Test set

In [0]:
X_tr = X
X_te = test_X
y_tr = y
y_te = test_y

In [0]:
# Code for if the train or test sets need to be shuffled

# train set
# import random
# random.seed(23)
# c = list(zip(X_tr, y_tr))
# random.shuffle(c)
# X_tr, y_tr = zip(*c)

# test set
# import random
# random.seed(23)
# c = list(zip(X_te, y_te))
# random.shuffle(c)
# X_te, y_te = zip(*c)

In [0]:
!pip install sklearn_crfsuite

The Model

In [0]:
from sklearn_crfsuite import CRF

In [0]:
crf = CRF(algorithm='lbfgs',
          c1=10,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [0]:
crf.fit(X_tr, y_tr)
y_pred = crf.predict(X_te)

Evaluation

In [0]:
from nervaluate import Evaluator
from sklearn_crfsuite import metrics

In [0]:
print(metrics.flat_classification_report(y_te, y_pred))

In [0]:
test_sents_labels = []
for sentence in test_sentences:
    sentence = [token[2] for token in sentence]
    test_sents_labels.append(sentence)

In [0]:
evaluator = Evaluator(y_te, y_pred, tags= [''], loader='list')

In [0]:
results, results_per_tag = evaluator.evaluate()

In [0]:
results